<a href="https://colab.research.google.com/github/kesanir/ML-AI-TRADING/blob/main/OPTIONS_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Step 1: Download options data for the previous day
def download_options_data(symbol, date):
    # Get the date for the previous trading day
    prev_date = date - timedelta(days=1)
    while prev_date.weekday() > 4:  # Skip weekend
        prev_date -= timedelta(days=1)

    # Fetch options data
    ticker = yf.Ticker(symbol)
    options = ticker.option_chain(prev_date.strftime('%Y-%m-%d'))

    # Combine calls and puts into one DataFrame
    options_chain = pd.concat([options.calls, options.puts])
    options_chain['expirationDate'] = pd.to_datetime(options_chain['expirationDate'])

    return options_chain

# Step 2: Analyze data for potential trades
def analyze_options(options_data):
    # Here's a very basic analysis:
    # - Look for options with high open interest but low implied volatility
    # - This is a simplified model; real analysis would be much more sophisticated
    options_data['implied_volatility'] = options_data['impliedVolatility']  # Just renaming for clarity

    # Filter for options with high open interest and relatively low implied volatility
    potential_trades = options_data[(options_data['openInterest'] > options_data['openInterest'].quantile(0.75)) &
                                    (options_data['implied_volatility'] < options_data['implied_volatility'].quantile(0.5))]

    # Sort by potential return (this is very simplistic; might not correlate with actual profit)
    potential_trades = potential_trades.sort_values('lastPrice', ascending=True)
    return potential_trades[['contractSymbol', 'strike', 'lastPrice', 'openInterest', 'implied_volatility', 'expirationDate']]

# Main function to run the analysis
def main():
    symbol = "TSLA"  # Example stock symbol
    today = datetime.now().date()

    # Download options data
    options_data = download_options_data(symbol, today)

    if options_data.empty:
        print("No options data available for the given date.")
        return

    # Analyze the data
    trade_opportunities = analyze_options(options_data)

    # Display top 5 potential trades
    print(trade_opportunities.head())

if __name__ == "__main__":
    main()

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.